In [ ]:
from PIL import Image, ImageEnhance
import os
import io
from rembg import remove
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

dataset_dir = 'path'
output_dir = 'path'
target_size = (3000, 3000)

def remove_bg_white_resize_brightness(image_path, factor=1.2):
    with Image.open(image_path).convert('RGBA') as img:
        img_bytes = io.BytesIO()
        img.save(img_bytes, format="PNG")
        img_bytes = img_bytes.getvalue()
        output_data = remove(img_bytes)
        result_image = Image.open(io.BytesIO(output_data)).convert('RGBA')
        white_bg = Image.new("RGBA", result_image.size, (255, 255, 255, 255))
        white_bg.paste(result_image, mask=result_image.split()[3])
        white_bg = white_bg.resize(target_size, Image.Resampling.LANCZOS)
        rgb_img = white_bg.convert("RGB")
        enhancer = ImageEnhance.Brightness(rgb_img)
        enhanced = enhancer.enhance(factor)
        return enhanced

def process_dataset(dataset_dir, output_dir, factor=1.2):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for class_name in os.listdir(dataset_dir):
        class_folder = os.path.join(dataset_dir, class_name)
        if os.path.isdir(class_folder):
            output_class_folder = os.path.join(output_dir, class_name)
            os.makedirs(output_class_folder, exist_ok=True)
            for filename in os.listdir(class_folder):
                if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_path = os.path.join(class_folder, filename)
                    processed_image = remove_bg_white_resize_brightness(image_path, factor)
                    output_path = os.path.join(output_class_folder, f"{os.path.splitext(filename)[0]}.jpg")
                    processed_image.save(output_path, "JPEG")
                    print(f"✅ Saved: {output_path}")

process_dataset(dataset_dir, output_dir, factor=1.2)
